In [1]:
import pandas as pd
import numpy as np
import pickle
from pandas import Timestamp
from collections import Counter
from sklearn.utils import shuffle

In [2]:
# ## load data

# # prediction features
# sessions_full = pickle.load(open('../Data/sessions.pkl', 'rb'))
# events_full_1 = pickle.load(open('../Data/event1.pkl', 'rb'))
# events_full_2 = pickle.load(open('../Data/event2.pkl', 'rb'))
# events_full = pd.concat([events_full_1, events_full_2])
# submission = pd.read_csv('../Data/sample_submission_2.csv')

# # training features and labels
# sessions_feature = pickle.load(open('../Data/sessions_feature.pkl', 'rb'))
# events_feature_1 = pickle.load(open('../Data/events_feature_1.pkl', 'rb'))
# events_feature_2 = pickle.load(open('../Data/events_feature_2.pkl', 'rb'))
# events_feature = pd.concat([events_feature_1, events_feature_2])
# training_label = pickle.load(open('../Data/training_label.pkl', 'rb'))

In [2]:
sessions = pickle.load(open('../Data/sessions.pkl', 'rb'))
events = pd.read_csv('../Data/events.csv')

## Feature Set for Prediction

In [3]:
# # use only the users in submission for higher performance
# pred_events_feature = events_full[events_full['user_id_hash'].isin(submission.user_id_hash)]
# pred_sessions_feature = sessions_full[sessions_full['user_id_hash'].isin(submission.user_id_hash)]

## Train_Val_Split

In [4]:
# training_label = shuffle(training_label)

In [5]:
# one_five = int(np.floor(len(training_label)/5))

In [6]:
# ## train and validation sets (features and labels seperately)

# # labels
# train_label_1 = training_label.iloc[:one_five*4]  # cv1
# val_label_1 = training_label.iloc[one_five*4:]

# train_label_2 = training_label.iloc[one_five:]  # cv2
# val_label_2 = training_label.iloc[:one_five]

# train_label_3 = pd.concat([training_label.iloc[:one_five], training_label.iloc[one_five*2:]])  # cv3
# val_label_3 = training_label.iloc[one_five:one_five*2]

# train_label_4 = pd.concat([training_label.iloc[:one_five*2], training_label.iloc[one_five*3:]])  # cv4
# val_label_4 = training_label.iloc[one_five*2:one_five*3]

# train_label_5 = pd.concat([training_label.iloc[:one_five*3], training_label.iloc[one_five*4:]])  # cv5
# val_label_5 = training_label.iloc[one_five*3:one_five*4]

# # features (drop the ones that's not in labels, that is, users who don't have any event in next 2 weeks)
# train_events_feature_1 = events_feature[events_feature['user_id_hash'].isin(train_label_1.user_id_hash)]  # cv1
# train_sessions_feature_1 = events_feature[events_feature['user_id_hash'].isin(train_label_1.user_id_hash)]
# val_events_feature_1 = events_feature[events_feature['user_id_hash'].isin(val_label_1.user_id_hash)]
# val_sessions_feature_1 = events_feature[events_feature['user_id_hash'].isin(val_label_1.user_id_hash)]

# train_events_feature_2 = events_feature[events_feature['user_id_hash'].isin(train_label_2.user_id_hash)]  # cv2
# train_sessions_feature_2 = events_feature[events_feature['user_id_hash'].isin(train_label_2.user_id_hash)]
# val_events_feature_2 = events_feature[events_feature['user_id_hash'].isin(val_label_2.user_id_hash)]
# val_sessions_feature_2 = events_feature[events_feature['user_id_hash'].isin(val_label_2.user_id_hash)]

# train_events_feature_3 = events_feature[events_feature['user_id_hash'].isin(train_label_3.user_id_hash)]  # cv3
# train_sessions_feature_3 = events_feature[events_feature['user_id_hash'].isin(train_label_3.user_id_hash)]
# val_events_feature_3 = events_feature[events_feature['user_id_hash'].isin(val_label_3.user_id_hash)]
# val_sessions_feature_3 = events_feature[events_feature['user_id_hash'].isin(val_label_3.user_id_hash)]

# train_events_feature_4 = events_feature[events_feature['user_id_hash'].isin(train_label_4.user_id_hash)]  # cv4
# train_sessions_feature_4 = events_feature[events_feature['user_id_hash'].isin(train_label_4.user_id_hash)]
# val_events_feature_4 = events_feature[events_feature['user_id_hash'].isin(val_label_4.user_id_hash)]
# val_sessions_feature_4 = events_feature[events_feature['user_id_hash'].isin(val_label_4.user_id_hash)]

# train_events_feature_5 = events_feature[events_feature['user_id_hash'].isin(train_label_5.user_id_hash)]  # cv5
# train_sessions_feature_5 = events_feature[events_feature['user_id_hash'].isin(train_label_5.user_id_hash)]
# val_events_feature_5 = events_feature[events_feature['user_id_hash'].isin(val_label_5.user_id_hash)]
# val_sessions_feature_5 = events_feature[events_feature['user_id_hash'].isin(val_label_5.user_id_hash)]

## Faeture Engineering

In [108]:
def avg_diff(ls, seven_endtime, fourteen_endtime, last=0, last2=0):
    '''calculate average intervals'''
    if len(ls) in [0,1]: return -1
    if last:
        ls2 = [e for e in ls if e > seven_endtime]
        return avg_diff(ls2, seven_endtime, fourteen_endtime)
    if last2:
        ls2 = [e for e in ls if e > fourteen_endtime]
        return avg_diff(ls2, seven_endtime, fourteen_endtime)
    return sum([(ls[i+1] - ls[i]).total_seconds()/3600 for i in range(len(ls)-1)])/(len(ls)-1)    

In [130]:
def compute_event_features(events):
    '''compute featuers from events'''
    
    
    events = events.drop('app_id', axis=1)
    if type(events.event_timestamp.iloc[0]) != Timestamp:
        events['event_timestamp'] = pd.to_datetime(events['event_timestamp'],unit='ms')
    events = events.sort_values(by='event_timestamp')
    events = events.reset_index(drop=True)
    
    
    user_id_hash = events.groupby('session_id')["user_id_hash"].first().reset_index(name="user_id_hash")
    # number of events in each session
    num_events = events.groupby('session_id')["event"].count().reset_index(name="event_count")
    # duration = last event - first event
    first_event = events.groupby('session_id')['event_timestamp'].first().reset_index(name="first_event")
    last_event = events.groupby('session_id')['event_timestamp'].last().reset_index(name="last_event")
    duration = first_event.join(last_event.set_index('session_id'), on='session_id')
    duration['duration'] = duration['last_event'] - duration['first_event']
    duration['duration'] = [e.total_seconds()/3600 for e in duration.duration]  # in hours
    
    
    event_endtime = events.event_timestamp.iloc[-1]
    seven_endtime = pd.to_datetime(event_endtime) - pd.Timedelta(days=7)
    fourteen_endtime = pd.to_datetime(event_endtime) - pd.Timedelta(days=14)
    per_session = user_id_hash.merge(num_events,on='session_id').merge(duration,on='session_id')

    # time spent on the app in hours
    total_time = per_session.groupby('user_id_hash')['duration'].sum().reset_index(name='total_time')
    total_time_last_week = per_session[per_session['first_event']>seven_endtime].\
                            groupby('user_id_hash')['duration'].sum().reset_index(name='total_time_last_week')
    total_time_last_two_week = per_session[per_session['first_event']>fourteen_endtime].\
                            groupby('user_id_hash')['duration'].sum().reset_index(name='total_time_last_two_week')

    # avg session duration
    avg_session_duration = per_session.groupby('user_id_hash')['duration'].mean().reset_index(name='avg_session_duration')
    avg_session_duration_last_week = per_session[per_session['first_event']>seven_endtime].\
                                        groupby('user_id_hash')['duration'].mean().\
                                        reset_index(name='avg_session_duration_last_week')
    avg_session_duration_last_two_week = per_session[per_session['first_event']>fourteen_endtime].\
                                        groupby('user_id_hash')['duration'].mean().\
                                        reset_index(name='avg_session_duration_last_two_week')

    # avg count of events in each session
    avg_event_count = per_session.groupby('user_id_hash')['event_count'].mean().reset_index(name='avg_event_count')

    # sum of purchase
    sum_purchase_value = events[events['event']=='8'].groupby('user_id_hash')['event_value'].\
                            sum().reset_index(name='sum_purchase_value')
    sum_purchase_value_last_week = events[(events['event']=='8') & (events['event_timestamp']>seven_endtime)].\
                            groupby('user_id_hash')['event_value'].\
                            sum().reset_index(name='sum_purchase_value_last_week')
    sum_purchase_value_last_two_week = events[(events['event']=='8') & (events['event_timestamp']>fourteen_endtime)].\
                            groupby('user_id_hash')['event_value'].\
                            sum().reset_index(name='sum_purchase_value_last_two_week')

    # number of purchase
    purchase_count = events[events['event']=='8'].groupby('user_id_hash')['event_value'].\
                            count().reset_index(name='purchase_count')
    purchase_count_last_week = events[(events['event']=='8') & (events['event_timestamp']>seven_endtime)].\
                            groupby('user_id_hash')['event_value'].\
                            count().reset_index(name='purchase_count_last_week')
    purchase_count_last_two_week = events[(events['event']=='8') & (events['event_timestamp']>fourteen_endtime)].\
                            groupby('user_id_hash')['event_value'].\
                            count().reset_index(name='purchase_count_last_two_week')
    
    
    event_features = total_time.merge(total_time_last_week, how='left', on='user_id_hash').\
                    merge(total_time_last_two_week, how='left', on='user_id_hash').\
                    merge(avg_session_duration, on='user_id_hash').\
                    merge(avg_session_duration_last_week, how='left', on='user_id_hash').\
                    merge(avg_session_duration_last_two_week, how='left', on='user_id_hash').\
                    merge(avg_event_count, on='user_id_hash').\
                    merge(sum_purchase_value, how='left', on='user_id_hash').\
                    merge(sum_purchase_value_last_week, how='left', on='user_id_hash').\
                    merge(sum_purchase_value_last_two_week, how='left', on='user_id_hash').\
                    merge(purchase_count, how='left', on='user_id_hash').\
                    merge(purchase_count_last_week, how='left', on='user_id_hash').\
                    merge(purchase_count_last_two_week, how='left', on='user_id_hash').fillna(0)
    
    
    # avg interval between purchases in hours
    purchase_events = events[events['event']=='8'][['event_timestamp', 'user_id_hash']].groupby('user_id_hash').\
                        agg({'event_timestamp': lambda x: list(x)})
    
    
    purchase_events['purchase_interval'] = purchase_events.event_timestamp.\
                                            apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime))
    purchase_events['purchase_interval_last_week'] = purchase_events.event_timestamp.\
                                                    apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime, 1, 0))
    purchase_events['purchase_interval_last_two_week'] = purchase_events.event_timestamp.\
                                                        apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime, 0, 1))
    purchase_events = purchase_events.drop('event_timestamp', axis=1)
    event_features = event_features.merge(purchase_events, how='left', on='user_id_hash').fillna(-1)
    
    
    return event_features

In [110]:
ev = compute_event_features(pd.read_csv('../Data/events.csv'))
print(len(ev))
ev.head()

621001


,user_id_hash,total_time,total_time_last_week,total_time_last_two_week,avg_session_duration,avg_session_duration_last_week,avg_session_duration_last_two_week,avg_event_count,sum_purchase_value,sum_purchase_value_last_week,sum_purchase_value_last_two_week,purchase_count,purchase_count_last_week,purchase_count_last_two_week,purchase_interval,purchase_interval_last_week,purchase_interval_last_two_week
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,7.434159,0.499168,0.499168,0.464635,0.499168,0.499168,37.312500,23.044,0.0,0.0,8.0,0.0,0.0,4.529346,-1.0,-1.0
1,000059859ec188af6035870faf885c3038cedda05b3a54...,2.497187,0.000000,0.000000,0.227017,0.000000,0.000000,26.363636,1.393,0.0,0.0,1.0,0.0,0.0,-1.000000,-1.0,-1.0
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,5.892048,0.000000,0.000000,0.310108,0.000000,0.000000,29.421053,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,2.490035,0.000000,0.000000,0.355719,0.000000,0.000000,27.000000,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0.473771,0.000000,0.000000,0.157924,0.000000,0.000000,14.000000,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0


# Features from Sessions

In [57]:
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,is_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000,25837591,1538874289458,False,True,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2201961907282901522,1543712977293,Asia/Manila,28800000,35050130,1538874289458,False,True,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,18781111175537580,1539215568666,Asia/Manila,28800000,11343848,1538874289458,False,True,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,1477540082628742048,1540120743010,Asia/Manila,28800000,13499724,1538874289458,False,True,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,8184875317380844086,1542671625528,Asia/Manila,28800000,32788010,1538874289458,False,True,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [82]:
sessions = sessions.drop(['timezone_offset', 'timezone', 'is_user_first_session', 'is_session', 'city', 'region',
                          'latitude', 'longitude', 'locale', 'session_index', 'device_id'], axis=1)

In [83]:
sessions = sessions.dropna().reset_index(drop=True)

In [87]:
sessions['start_timestamp'] = pd.to_datetime(sessions['start_timestamp'], unit='ms')
sessions['user_created_timestamp'] = pd.to_datetime(sessions['user_created_timestamp'], unit='ms')
sessions = sessions.sort_values(by='start_timestamp')
sessions = sessions.reset_index(drop=True)

In [88]:
sessions.head()

,session_id,start_timestamp,previous_sessions_duration,user_created_timestamp,country,os_name,user_id_hash
0,5558845121177764917,2018-11-14 17:09:24.580,25837591,2018-10-07 01:04:49.458,PH,Android OS,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2201961907282901522,2018-12-02 01:09:37.293,35050130,2018-10-07 01:04:49.458,PH,Android OS,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,18781111175537580,2018-10-10 23:52:48.666,11343848,2018-10-07 01:04:49.458,PH,Android OS,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,1477540082628742048,2018-10-21 11:19:03.010,13499724,2018-10-07 01:04:49.458,PH,Android OS,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,8184875317380844086,2018-11-19 23:53:45.528,32788010,2018-10-07 01:04:49.458,PH,Android OS,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [102]:
event_endtime = events.event_timestamp.iloc[-1]
seven_endtime = pd.to_datetime(event_endtime) - pd.Timedelta(days=7)
fourteen_endtime = pd.to_datetime(event_endtime) - pd.Timedelta(days=14)

def avg_diff(ls, last=0, last2=0):
    if len(ls) in [0,1]: return -1
    if last:
        ls2 = [e for e in ls if e > seven_endtime]
        return avg_diff(ls2)
    if last2:
        ls2 = [e for e in ls if e > fourteen_endtime]
        return avg_diff(ls2)
    return sum([(ls[i+1] - ls[i]).total_seconds()/3600 for i in range(len(ls)-1)])/(len(ls)-1)

In [92]:
session_timestamp = sessions.groupby('user_id_hash').agg({'start_timestamp': lambda x: list(x)})

In [93]:
sessions = sessions.groupby('user_id_hash').agg({'user_created_timestamp': lambda x: x.iloc[0],
                                                'country': lambda x: x.iloc[0],
                                                'os_name': lambda x: x.iloc[0]}).\
                                            merge(session_timestamp, on='user_id_hash')

In [94]:
sessions.head()

,user_created_timestamp,country,os_name,start_timestamp
user_id_hash,,,,
00000db35682058bb7916cb90f85709f54c1a0f7a3b6de247d94b1bc20c36b97,2018-11-10 07:00:34.391,AU,iOS,"[2018-11-10 23:41:06.113000, 2018-12-13 11:31:..."
000059859ec188af6035870faf885c3038cedda05b3a5480a8223649629d951e,2018-11-22 19:10:12.526,US,Android OS,"[2018-11-27 10:51:41.250000, 2018-11-22 19:10:..."
000062e9be78f3da274fec338e78f89d12000e781967f23e248af3904cb5b3b7,2018-10-07 17:04:18.435,US,Android OS,"[2018-10-10 03:23:11.885000, 2018-10-11 01:46:..."
00006ad44c0d33a2a526caf75514643639b7b7070dfffd6d5abdb9e4f8099108,2018-10-18 03:48:45.789,ID,Android OS,"[2018-10-19 11:10:25.486000, 2018-10-19 22:50:..."
000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3035fbf5496aab9be7c,2018-11-22 20:34:45.415,US,Android OS,"[2018-12-01 10:04:58.057000, 2018-11-23 01:43:..."


In [95]:
len(sessions)

619833

In [103]:
sessions['elapse_since_last_session'] = [(Timestamp('2018-12-14 23:59:59.000000')-e[-1]).total_seconds()/3600 for e in sessions.start_timestamp]
sessions['avg_session_interval'] = sessions.start_timestamp.\
                                    apply(avg_diff(x, seven_endtime, fourteen_endtime))
sessions['avg_session_interval_last_week'] = sessions.start_timestamp.\
                                                apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime, 1, 0))
sessions['avg_session_interval_last_two_week'] = sessions.start_timestamp.\
                                                    apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime, 0, 1))
sessions['since_register'] = [(Timestamp('2018-12-14 23:59:59.000000')-s).total_seconds()/3600 for e,s in zip(sessions.start_timestamp, sessions.user_created_timestamp)]

In [104]:
sessions.head()

,user_created_timestamp,country,os_name,start_timestamp,elapse_since_last_session,avg_session_interval,since_register,avg_session_interval_last_week,avg_session_interval_last_two_week
user_id_hash,,,,,,,,,
00000db35682058bb7916cb90f85709f54c1a0f7a3b6de247d94b1bc20c36b97,2018-11-10 07:00:34.391,AU,iOS,"[2018-11-10 23:41:06.113000, 2018-12-13 11:31:...",796.399776,0.905223,816.314691,-1.0,-1.0
000059859ec188af6035870faf885c3038cedda05b3a5480a8223649629d951e,2018-11-22 19:10:12.526,US,Android OS,"[2018-11-27 10:51:41.250000, 2018-11-22 19:10:...",513.420322,-8.389278,421.138264,-1.0,-1.0
000062e9be78f3da274fec338e78f89d12000e781967f23e248af3904cb5b3b7,2018-10-07 17:04:18.435,US,Android OS,"[2018-10-10 03:23:11.885000, 2018-10-11 01:46:...",1629.049837,-1.729884,1580.613088,-1.0,-1.0
00006ad44c0d33a2a526caf75514643639b7b7070dfffd6d5abdb9e4f8099108,2018-10-18 03:48:45.789,ID,Android OS,"[2018-10-19 11:10:25.486000, 2018-10-19 22:50:...",1388.187003,-3.920128,1356.825976,-1.0,-1.0
000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3035fbf5496aab9be7c,2018-11-22 20:34:45.415,US,Android OS,"[2018-12-01 10:04:58.057000, 2018-11-23 01:43:...",531.420440,-51.375878,325.916929,-1.0,-1.0


In [131]:
def compute_session_features(sessions):
    '''compute features from sessions'''
    
    
    sessions = sessions.drop(['timezone_offset', 'timezone', 'is_user_first_session', 'is_session', 'city', 'region',
                            'latitude', 'longitude', 'locale', 'session_index', 'device_id'], axis=1).dropna()
    if type(sessions.start_timestamp.iloc[0]) != Timestamp:
        sessions['start_timestamp'] = pd.to_datetime(sessions['start_timestamp'], unit='ms')
    sessions['user_created_timestamp'] = pd.to_datetime(sessions['user_created_timestamp'], unit='ms')
    sessions = sessions.sort_values(by='start_timestamp')
    sessions = sessions.reset_index(drop=True)
    
    
    session_endtime = sessions.start_timestamp.iloc[-1]
    seven_endtime = pd.to_datetime(session_endtime) - pd.Timedelta(days=7)
    fourteen_endtime = pd.to_datetime(session_endtime) - pd.Timedelta(days=14)
    
    
    session_timestamp = sessions.groupby('user_id_hash').agg({'start_timestamp': lambda x: list(x)})
    user_sessions = sessions.groupby('user_id_hash').agg({'user_created_timestamp': lambda x: x.iloc[0],
                                                         'country': lambda x: x.iloc[0],
                                                         'os_name': lambda x: x.iloc[0]}).\
                                                     merge(session_timestamp, on='user_id_hash')
    user_sessions['elapse_since_last_session'] = [(session_endtime-e[-1]).total_seconds()/3600 for e in user_sessions.start_timestamp]
    user_sessions['avg_session_interval'] = user_sessions.start_timestamp.\
                                        apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime))
    user_sessions['avg_session_interval_last_week'] = user_sessions.start_timestamp.\
                                                    apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime, 1, 0))
    user_sessions['avg_session_interval_last_two_week'] = user_sessions.start_timestamp.\
                                                        apply(lambda x: avg_diff(x, seven_endtime, fourteen_endtime, 0, 1))
    user_sessions['since_register'] = [(session_endtime-e).total_seconds()/3600 for e in user_sessions.user_created_timestamp]
    
    
    return user_sessions

In [116]:
se = compute_session_features(pickle.load(open('../Data/sessions.pkl', 'rb')))
print(len(se))
se.head()

619833


,user_created_timestamp,country,os_name,start_timestamp,elapse_since_last_session,avg_session_interval,avg_session_interval_last_week,avg_session_interval_last_two_week,since_register
user_id_hash,,,,,,,,,
00000db35682058bb7916cb90f85709f54c1a0f7a3b6de247d94b1bc20c36b97,2018-11-10 07:00:34.391,AU,iOS,"[2018-11-10 07:00:34.391000, 2018-11-10 07:53:...",36.468411,36.205541,-1.0,-1.0,832.990304
000059859ec188af6035870faf885c3038cedda05b3a5480a8223649629d951e,2018-11-22 19:10:12.526,US,Android OS,"[2018-11-22 19:10:12.526000, 2018-11-22 20:29:...",421.138398,10.153756,-1.0,-1.0,532.829711
000062e9be78f3da274fec338e78f89d12000e781967f23e248af3904cb5b3b7,2018-10-07 17:04:18.435,US,Android OS,"[2018-10-07 17:04:18.435000, 2018-10-07 17:04:...",1456.093100,6.529820,-1.0,-1.0,1638.928069
00006ad44c0d33a2a526caf75514643639b7b7070dfffd6d5abdb9e4f8099108,2018-10-18 03:48:45.789,ID,Android OS,"[2018-10-18 03:48:45.789000, 2018-10-18 05:44:...",1118.540208,33.705866,-1.0,-1.0,1388.187138
000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3035fbf5496aab9be7c,2018-11-22 20:34:45.415,US,Android OS,"[2018-11-22 20:34:45.415000, 2018-11-22 20:34:...",325.917063,51.375878,-1.0,-1.0,531.420575


## X_train, y_train

In [124]:
# get y_train
events = pd.read_csv('../Data/events.csv')
events['event_timestamp'] = pd.to_datetime(events['event_timestamp'],unit='ms')
events = events.sort_values(by='event_timestamp')
events = events.reset_index(drop=True)

event_end_time = events['event_timestamp'][events.shape[0]-1] - pd.Timedelta(days=14)
events_label = events[events['event_timestamp'] > event_end_time]
events_label.loc['purchases'] = list(events_label['event'].eq('8'))

seven_endtime = pd.to_datetime(event_end_time) + pd.Timedelta(days=7)
fourteen_endtime = pd.to_datetime(event_end_time) + pd.Timedelta(days=14)
events_7 = events_label[np.logical_and(events_label['event_timestamp'] > event_end_time, 
                                       events_label['event_timestamp'] <= seven_endtime)].\
            groupby('user_id_hash')['purchases'].sum().reset_index(name='purchase_in_7_days')
events_14 = events_label[events_label['event_timestamp'] > event_end_time].\
            groupby('user_id_hash')['purchases'].sum().reset_index(name='purchase_in_14_days')

y_train = events_14.merge(events_7, how='left', on='user_id_hash').fillna(0)
y_train['purchase_in_7_days'] = [1 if e !=0 else 0 for e in y_train.purchase_in_7_days]
y_train['purchase_in_14_days'] = [1 if e !=0 else 0 for e in y_train.purchase_in_14_days]

In [125]:
y_train.head()

,user_id_hash,purchase_in_14_days,purchase_in_7_days
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,0,0
1,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0,0
2,0000a464ad921c3288e73a3a788b5f113cabf84251a74b...,0,0
3,0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87...,0,0
4,00020102f66f89759868c843ae9fe634870f132f9e3d79...,0,0


In [126]:
Counter(y_train.purchase_in_14_days)

Counter({0: 83165, 1: 6126})

In [135]:
## get X_train


# events_X
events_data = events[events['event_timestamp'] <= event_end_time]
events_data = events_data[events_data['user_id_hash'].isin(y_train.user_id_hash)]
events_X = compute_event_features(events_data)


# sessions_X
sessions = pickle.load(open('../Data/sessions.pkl', 'rb'))
sessions['start_timestamp'] = pd.to_datetime(sessions['start_timestamp'],unit='ms')
sessions = sessions.sort_values(by='start_timestamp')
sessions = sessions.reset_index(drop=True)

session_end_date = sessions['start_timestamp'][sessions.shape[0]-1] - pd.Timedelta(days=14)
sessions_data = sessions[sessions['start_timestamp'] <= session_end_date]
sessions_data = sessions_data[sessions_data['user_id_hash'].isin(y_train.user_id_hash)]
sessions_X = compute_session_features(sessions_data)

In [133]:
events_X.head()

,user_id_hash,total_time,total_time_last_week,total_time_last_two_week,avg_session_duration,avg_session_duration_last_week,avg_session_duration_last_two_week,avg_event_count,sum_purchase_value,sum_purchase_value_last_week,sum_purchase_value_last_two_week,purchase_count,purchase_count_last_week,purchase_count_last_two_week,purchase_interval,purchase_interval_last_week,purchase_interval_last_two_week
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,6.934991,0.000000,0.000000,0.462333,0.000000,0.000000,33.866667,23.044,0.0,0.0,8.0,0.0,0.0,4.529346,-1.0,-1.0
1,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0.473771,0.000000,0.473771,0.236885,0.000000,0.236885,20.500000,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0
2,0000a464ad921c3288e73a3a788b5f113cabf84251a74b...,0.285922,0.000000,0.285922,0.142961,0.000000,0.142961,13.500000,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0
3,0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87...,1.466552,1.466552,1.466552,0.183319,0.183319,0.183319,15.875000,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0
4,00020102f66f89759868c843ae9fe634870f132f9e3d79...,0.099715,0.099715,0.099715,0.099715,0.099715,0.099715,16.000000,0.000,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.0,-1.0


In [134]:
len(events_X)

88621

In [136]:
sessions_X.head()

,user_created_timestamp,country,os_name,start_timestamp,elapse_since_last_session,avg_session_interval,avg_session_interval_last_week,avg_session_interval_last_two_week,since_register
user_id_hash,,,,,,,,,
00000db35682058bb7916cb90f85709f54c1a0f7a3b6de247d94b1bc20c36b97,2018-11-10 07:00:34.391,AU,iOS,"[2018-11-10 07:00:34.391000, 2018-11-10 07:53:...",326.751184,8.106624,-1.000000,-1.000000,496.990298
000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3035fbf5496aab9be7c,2018-11-22 20:34:45.415,US,Android OS,"[2018-11-22 20:34:45.415000, 2018-11-22 20:34:...",169.228539,8.730677,-1.000000,8.730677,195.420569
0000a464ad921c3288e73a3a788b5f113cabf84251a74b269e32da9110e8bca3,2018-11-22 06:13:36.656,US,iOS,"[2018-11-22 06:13:36.656000, 2018-11-23 01:27:...",190.540230,19.232772,-1.000000,19.232772,209.773002
0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87611492c62e2e6a9540,2018-11-26 08:00:07.319,RO,Android OS,"[2018-11-26 08:00:11.988000, 2018-11-26 09:33:...",62.554608,6.180239,6.180239,6.180239,111.997818
00020102f66f89759868c843ae9fe634870f132f9e3d7990ca9c794a564477be,2018-11-30 20:14:16.049,GB,iOS,[2018-11-30 20:14:16.049000],3.762060,-1.000000,-1.000000,-1.000000,3.762060


In [137]:
len(sessions_X)

88521

In [138]:
# combine to get X_train
X_train = events_X.merge(sessions_X, how='inner', on='user_id_hash')

In [139]:
len(X_train)

88507

In [140]:
len(y_train)

89291

In [142]:
X_train = X_train.drop(['user_created_timestamp','country','os_name','start_timestamp'],axis=1)

In [144]:
training_set = X_train.merge(y_train, how='left', on='user_id_hash')

In [145]:
training_set.head()

,user_id_hash,total_time,total_time_last_week,total_time_last_two_week,avg_session_duration,avg_session_duration_last_week,avg_session_duration_last_two_week,avg_event_count,sum_purchase_value,sum_purchase_value_last_week,...,purchase_interval,purchase_interval_last_week,purchase_interval_last_two_week,elapse_since_last_session,avg_session_interval,avg_session_interval_last_week,avg_session_interval_last_two_week,since_register,purchase_in_14_days,purchase_in_7_days
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,6.934991,0.000000,0.000000,0.462333,0.000000,0.000000,33.866667,23.044,0.0,...,4.529346,-1.0,-1.0,326.751184,8.106624,-1.000000,-1.000000,496.990298,0,0
1,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0.473771,0.000000,0.473771,0.236885,0.000000,0.236885,20.500000,0.000,0.0,...,-1.000000,-1.0,-1.0,169.228539,8.730677,-1.000000,8.730677,195.420569,0,0
2,0000a464ad921c3288e73a3a788b5f113cabf84251a74b...,0.285922,0.000000,0.285922,0.142961,0.000000,0.142961,13.500000,0.000,0.0,...,-1.000000,-1.0,-1.0,190.540230,19.232772,-1.000000,19.232772,209.773002,0,0
3,0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87...,1.466552,1.466552,1.466552,0.183319,0.183319,0.183319,15.875000,0.000,0.0,...,-1.000000,-1.0,-1.0,62.554608,6.180239,6.180239,6.180239,111.997818,0,0
4,00020102f66f89759868c843ae9fe634870f132f9e3d79...,0.099715,0.099715,0.099715,0.099715,0.099715,0.099715,16.000000,0.000,0.0,...,-1.000000,-1.0,-1.0,3.762060,-1.000000,-1.000000,-1.000000,3.762060,0,0


In [148]:
training_set.iloc[:,1:-2]

,total_time,total_time_last_week,total_time_last_two_week,avg_session_duration,avg_session_duration_last_week,avg_session_duration_last_two_week,avg_event_count,sum_purchase_value,sum_purchase_value_last_week,sum_purchase_value_last_two_week,...,purchase_count_last_week,purchase_count_last_two_week,purchase_interval,purchase_interval_last_week,purchase_interval_last_two_week,elapse_since_last_session,avg_session_interval,avg_session_interval_last_week,avg_session_interval_last_two_week,since_register
0,6.934991,0.000000,0.000000,0.462333,0.000000,0.000000,33.866667,23.044000,0.000,0.000,...,0.0,0.0,4.529346,-1.000000,-1.000000,326.751184,8.106624,-1.000000,-1.000000,496.990298
1,0.473771,0.000000,0.473771,0.236885,0.000000,0.236885,20.500000,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,169.228539,8.730677,-1.000000,8.730677,195.420569
2,0.285922,0.000000,0.285922,0.142961,0.000000,0.142961,13.500000,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,190.540230,19.232772,-1.000000,19.232772,209.773002
3,1.466552,1.466552,1.466552,0.183319,0.183319,0.183319,15.875000,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,62.554608,6.180239,6.180239,6.180239,111.997818
4,0.099715,0.099715,0.099715,0.099715,0.099715,0.099715,16.000000,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,3.762060,-1.000000,-1.000000,-1.000000,3.762060
5,1.758614,1.758614,1.758614,0.351723,0.351723,0.351723,27.800000,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,0.055755,8.857904,8.857904,8.857904,44.345278
6,13.268400,0.000000,7.432899,0.301555,0.000000,0.285881,22.522727,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,173.082300,7.462216,-1.000000,3.810787,695.437413
7,43.756814,5.217313,19.218381,0.361627,0.289851,0.343185,23.504132,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,7.673076,4.003397,8.179413,5.329350,632.209291
8,12.575390,3.827089,8.373339,0.273378,0.478386,0.380606,18.652174,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,19.422591,10.506267,18.297356,12.081540,583.158603
9,0.719847,0.000000,0.719847,0.719847,0.000000,0.719847,44.000000,0.000000,0.000,0.000,...,0.0,0.0,-1.000000,-1.000000,-1.000000,226.384016,7.300338,-1.000000,7.300338,233.684354


## Model

In [143]:
from sklearn.metrics import roc_curve, auc
import sys
# data visualization package
import matplotlib.pyplot as plt

# ignore the warning
import warnings
# for evaluation metric
# accuracy
from sklearn.metrics import accuracy_score
# Report
from sklearn.metrics import classification_report
# AUC
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from pylab import rcParams

# for model comparision
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier

In [149]:
rf_7 = RandomForestClassifier(n_estimators=100, max_depth =5, max_features = 'sqrt', max_leaf_nodes= 8).fit(training_set.iloc[:,1:-2], training_set['purchase_in_7_days'])
rf_14 = RandomForestClassifier(n_estimators=100, max_depth =5, max_features = 'sqrt', max_leaf_nodes= 8).fit(training_set.iloc[:,1:-2], training_set['purchase_in_14_days'])

In [151]:
events = pd.read_csv('../Data/events.csv')
sessions = pickle.load(open('../Data/sessions.pkl', 'rb'))

In [152]:
submission = pd.read_csv('../Data/sample_submission_2.csv')

In [153]:
events_test = events[events['user_id_hash'].isin(submission.user_id_hash)]
sessions_test = sessions[sessions['user_id_hash'].isin(submission.user_id_hash)]

In [155]:
events_X = compute_event_features(events_test)
sessions_X = compute_session_features(sessions_test)

In [157]:
X_test = events_X.merge(sessions_X, on='user_id_hash')

In [161]:
X_test = X_test.drop(['user_created_timestamp','country','os_name','start_timestamp'],axis=1)

In [175]:
X_test.head()

,user_id_hash,total_time,total_time_last_week,total_time_last_two_week,avg_session_duration,avg_session_duration_last_week,avg_session_duration_last_two_week,avg_event_count,sum_purchase_value,sum_purchase_value_last_week,...,purchase_interval,purchase_interval_last_week,purchase_interval_last_two_week,elapse_since_last_session,avg_session_interval,avg_session_interval_last_week,avg_session_interval_last_two_week,since_register,pred_7,pred_14
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,7.434159,0.499168,0.499168,0.464635,0.499168,0.499168,37.312500,23.044,0.0,...,4.529346,-1.0,-1.0,36.466749,36.205541,-1.0,-1.0,832.988641,0.891220,0.848168
1,000059859ec188af6035870faf885c3038cedda05b3a54...,2.497187,0.000000,0.000000,0.227017,0.000000,0.000000,26.363636,1.393,0.0,...,-1.000000,-1.0,-1.0,421.136736,10.153756,-1.0,-1.0,532.828048,0.970117,0.946072
2,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0.473771,0.000000,0.000000,0.157924,0.000000,0.000000,14.000000,0.000,0.0,...,-1.000000,-1.0,-1.0,325.915401,51.375878,-1.0,-1.0,531.418912,0.984848,0.978220
3,0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87...,1.466552,0.000000,0.000000,0.162950,0.000000,0.000000,14.222222,0.000,0.0,...,-1.000000,-1.0,-1.0,280.565016,18.603316,-1.0,-1.0,447.996161,0.984848,0.977330
4,0000c5de50d243cc39f6ac4268d7511705585ca3f2a20f...,0.634095,0.000000,0.000000,0.211365,0.000000,0.000000,22.000000,0.000,0.0,...,-1.000000,-1.0,-1.0,912.163312,99.295397,-1.0,-1.0,1110.786054,0.984786,0.978220


In [173]:
X_test['pred_7'] = rf_7.predict_proba(X_test.iloc[:,1:-2])
X_test['pred_14'] = rf_14.predict_proba(X_test.iloc[:,1:-2])

In [174]:
pred_7 = X_test[['user_id_hash','pred_7']]
pred_14 = X_test[['user_id_hash','pred_14']]

In [176]:
submit = submission.copy()
submit = submit.merge(pred_7, how='left', on='user_id_hash').merge(pred_14, how='left', on='user_id_hash').fillna(0)

In [179]:
submit = submit.drop(['user_purchase_binary_7_days', 'user_purchase_binary_14_days'],axis=1)

In [180]:
submit['pred_7'] = [1-e for e in submit.pred_7]
submit['pred_14'] = [1-e for e in submit.pred_14]

In [181]:
submit.head()

,user_id_hash,pred_7,pred_14
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.015152,0.021780
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.015411,0.022097
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.015152,0.021780
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.015484,0.023503
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.015152,0.021780


In [182]:
submit.to_csv('prediction.csv', index=False)